In [5]:
import threading
import os
import time

In [2]:
# concurrent/parallel execution
# concurrent means, dependencies exists
# parallel means no dependencies, we will discuss as we go


In [7]:
# simple thread and thread id


def worker():
    print("=== Inside worker thread ===")
    print("Thread name :", threading.current_thread().name)
    print("Thread ID   :", threading.get_ident())
    print("Process ID  :", os.getpid())
    time.sleep(10) # 10 seconds sleep, 

print("=== Inside main thread ===")
# every python program runs on thread as well, known as main thread
print("Thread name :", threading.current_thread().name)
print("Thread ID   :", threading.get_ident())
print("Process ID  :", os.getpid())

t = threading.Thread(target=worker, name="Worker-1")
t.start() # start new thread
print ("waiting thread to complete")
t.join() # wait for thread to complete
print (" thread completed")


=== Inside main thread ===
Thread name : MainThread
Thread ID   : 18704
Process ID  : 8356
=== Inside worker thread ===
Thread name : Worker-1
Thread ID   : 30640
Process ID  : 8356
waiting thread to complete
 thread completed


In [1]:
# concurrent access probelm

import threading
import time
import random

# Shared resource
restroom_credits = 1

def use_restroom(person_name):
    global restroom_credits

    print(f"{person_name} is trying to enter the restroom")

    # Check credit
    if restroom_credits > 0:
        print(f"{person_name} sees credit available")

        # Random delay to make race condition visible
        time.sleep(random.uniform(0.5, 2))

        restroom_credits -= 1
        print(f"{person_name} ENTERED restroom. Credits left: {restroom_credits}")

        # Simulate restroom usage
        time.sleep(random.uniform(1, 3))

        restroom_credits += 1
        print(f"{person_name} EXITED restroom. Credits restored: {restroom_credits}")
    else:
        print(f"{person_name} could NOT enter (no credits)")

# Create threads
person1 = threading.Thread(target=use_restroom, args=("Person1",))
person2 = threading.Thread(target=use_restroom, args=("Person2",))

# Start threads
person1.start()
person2.start()

# Wait for both to finish
person1.join()
person2.join()

print("Simulation complete")

Person1 is trying to enter the restroom
Person1 sees credit available
Person2 is trying to enter the restroom
Person2 sees credit available
Person2 ENTERED restroom. Credits left: 0
Person1 ENTERED restroom. Credits left: -1
Person1 EXITED restroom. Credits restored: 0
Person2 EXITED restroom. Credits restored: 1
Simulation complete


In [3]:
# with lock

import threading
import time
import random

restroom_credits = 1
lock = threading.Lock()

def use_restroom(person_name):
    global restroom_credits

    print(f"{person_name} is trying to enter the restroom")

    with lock:
        if restroom_credits > 0:
            print(f"{person_name} got the lock and ENTERED")

            restroom_credits -= 1
        else:
            print(f"{person_name} could NOT enter (no credits)")
            return

    # Restroom usage happens outside lock
    time.sleep(random.uniform(1, 3))

    with lock:
        restroom_credits += 1
        print(f"{person_name} EXITED restroom. Credits restored: {restroom_credits}")

# Threads
person1 = threading.Thread(target=use_restroom, args=("Person1",))
person2 = threading.Thread(target=use_restroom, args=("Person2",))

person1.start()
person2.start()

person1.join()
person2.join()

print("Simulation complete")

Person1 is trying to enter the restroom
Person1 got the lock and ENTERED
Person2 is trying to enter the restroom
Person2 could NOT enter (no credits)
Person1 EXITED restroom. Credits restored: 1
Simulation complete


In [4]:
# Producer Threads  --->  Queue  --->  Consumer Thread
""" payload example
{
  "order_id": 1001,
  "amount": 2499.50,
  "items_count": 3
}
"""

' payload example\n{\n  "order_id": 1001,\n  "amount": 2499.50,\n  "items_count": 3\n}\n'

In [5]:
import threading
import queue
import time
import random
import json

# Shared queue (thread-safe)
order_queue = queue.Queue()

# Sentinel to stop consumer
STOP_SIGNAL = object()

# -------------------------
# Producer (Order Creator)
# -------------------------
def producer(name, order_count):
    for _ in range(order_count):
        order = {
            "order_id": random.randint(1000, 9999),
            "amount": round(random.uniform(100, 5000), 2),
            "items_count": random.randint(1, 10)
        }

        print(f"[PRODUCER-{name}] Created order: {json.dumps(order)}")

        order_queue.put(order)

        # Random delay
        time.sleep(random.uniform(0.5, 2))

    print(f"[PRODUCER-{name}] Finished producing orders")


# -------------------------
# Consumer (Order Processor)
# -------------------------
def consumer():
    while True:
        order = order_queue.get()

        if order is STOP_SIGNAL:
            print("[CONSUMER] Stop signal received. Shutting down.")
            break

        print(f"[CONSUMER] Processing order {order['order_id']} "
              f"(Amount: {order['amount']}, Items: {order['items_count']})")

        # Simulate processing time
        time.sleep(random.uniform(1, 3))

        print(f"[CONSUMER] Completed order {order['order_id']}")

        order_queue.task_done()


# -------------------------
# Thread Setup
# -------------------------
producer1 = threading.Thread(target=producer, args=("A", 3))
producer2 = threading.Thread(target=producer, args=("B", 3))
consumer_thread = threading.Thread(target=consumer)

# Start threads
consumer_thread.start()
producer1.start()
producer2.start()

# Wait for producers to finish
producer1.join()
producer2.join()

# Wait until all orders are processed
order_queue.join()

# Stop consumer
order_queue.put(STOP_SIGNAL)
consumer_thread.join()
 

print("Order processing system shut down cleanly.")

[PRODUCER-A] Created order: {"order_id": 6334, "amount": 165.79, "items_count": 3}
[CONSUMER] Processing order 6334 (Amount: 165.79, Items: 3)
[PRODUCER-B] Created order: {"order_id": 6109, "amount": 3012.09, "items_count": 5}
[PRODUCER-A] Created order: {"order_id": 5715, "amount": 3632.22, "items_count": 2}
[PRODUCER-A] Created order: {"order_id": 5557, "amount": 3799.56, "items_count": 9}
[CONSUMER] Completed order 6334
[CONSUMER] Processing order 6109 (Amount: 3012.09, Items: 5)
[PRODUCER-B] Created order: {"order_id": 3164, "amount": 2745.24, "items_count": 1}
[PRODUCER-A] Finished producing orders
[CONSUMER] Completed order 6109
[CONSUMER] Processing order 5715 (Amount: 3632.22, Items: 2)
[PRODUCER-B] Created order: {"order_id": 7947, "amount": 1930.15, "items_count": 10}
[PRODUCER-B] Finished producing orders
[CONSUMER] Completed order 5715
[CONSUMER] Processing order 5557 (Amount: 3799.56, Items: 9)
[CONSUMER] Completed order 5557
[CONSUMER] Processing order 3164 (Amount: 2745.